In [ ]:
# In order to correctly run the notebook
#PYTHONPATH=PATH_TO_DIR/lmnav_code_release OPENAI_API_KEY=sk-1  jupyter notebook

In [ ]:
import base64
import json
import networkx
import numpy as np
import pickle
from IPython.display import display, Javascript, HTML

import lm_nav
from lm_nav.navigation_graph import NavigationGraph
from lm_nav import optimal_route, pipeline
import gdown

## Select if you are using small or large graph

In [ ]:
use_large_graph = True

In [ ]:
if use_large_graph:
    graph_file_gdrive_id = "1Cru_rHDKKymYid0zKAG6sqbo2jiGKBIs"
    all_routes = [
        (5, "Go straight toward the white building. Continue straight passing by a white truck until you reach a stop sign."),
        (5, "After passing a white building, take right next to a white truck. Then take left and go towards a square with a large tree. Go further, until you find a stop sign."),
        (173, "Start going around a building with a red-black wall and pass by a fire hydrant. Take a right and enter a grove. Continue straight and take a right, when you see a manhole cover. Go forward and left, and look for a trailer."),
        (108, "Take a right next to a stop sign. Look for a glass building, after passing by a white car."),
        (247, "Follow the road and take the right, you should see a blue semi-truck. Behind the truck, take a right next to an orange traffic cone. Go towards a blue dumpster and take left. Look for a picnic bench."),
        (70, "Go towards a white trailer. Then take left and go to the traffic lights. Take left again, and look for a traffic cone."),
        (215, "Go straight, passing by a stop sign and a  manhole cover. Next, you will see a disabled Parking spot and a red building."),
        (103, "First, you need to find a stop sign. Then take left and right and continue until you reach a square with a tree. Continue first straight, then right, until you find a white truck. The final destination is a white building."),
        (103, "Go to a stop sign. Continue straight, passing by a white truck. The final destination is a white building."),
        (211, "Go straight, until you find a glass building. Drive to a white car nearby. Drive to reach a stop sign, this is your destination.")
    ]
    all_routes_gt = [
        [5, 8, 77],
        [5, 8, 23, 261, 77],
        [173, 160, 150, 191, 129, 45],
        [108, 210, 217, 220],
        [247, 254, 264, 275],
        [70, 39, 34, 257],
        [215, 194, 184, 170],
        [103, 267, 22, 8],
        [103, 16, 8],
        [211, 220, 217, 204],
    ]
    landmarks_cache = eval("[['a white building', 'a white truck', 'a stop sign'], ['a white building', 'a white truck', 'a square with a large tree', 'a stop sign'], ['a building with a red-black wall', 'a fire hydrant', 'a grove', 'a manhole cover', 'a trailer'], ['a stop sign', 'a white car', 'a glass building'], ['a blue semi-truck', 'an orange traffic cone', 'a blue dumpster', 'a picnic bench'], ['a white trailer', 'traffic lights', 'a traffic cone'], ['a stop sign', 'a manhole cover', 'a disabled Parking spot', 'a red building'], ['a stop sign', 'a square with a tree', 'a white truck', 'a white building'], ['a stop sign', 'a white truck', 'a white building'], ['a glass building', 'a white car', 'a stop sign']]")
else:
    graph_file_gdrive_id = "1Ua8dWM8C-6iTVH1pDucJ5cNRWfyhTYJ4"
    all_routes = [
        (180, "Go straight towards a stop sign, take left and go until you reach a traffic cone. Take another left and then right going towards a blue box. From there take left and look for a baby stroller."),
        (215, "Go towards the blue box, take right and left until you reach a traffic cone. Take left and pass by a semi-truck until you find a big log."),
        (63, "Start at a traffic cone. Go towards a cardboard box and a parking lot. Continue driving, take a right, and pass by a picnic table. Take left and look for a stop sign."),
        (160, "Take first right towards a picnic table. Next, go to a square with a large tree, and take the left to another picnic table. Keep going until you reach a parking lot."),
        (61, "Go straight and take right next to a traffic cone. Go straight until you reach a parking lot. Take left, go through a lawn and look for a blue box."),
        (219, "Pass by a blue box and look for a big log. Take right and keep going straight, passing by a traffic cone. Take a right and finish at the parking lot."),
        (186, "Look for a traffic cone, take left and go straight until you find a square with a tree. Turn right, pass by a cardboard box and go to a picnic table."),
        (75, "Go straight pass a picnic table until you reach a street. Take right, pass by an orange trailer and take next right at an intersection. Next, take a right next to a traffic cone, take the next left, and pass by a baby stroller. Go straight and you will reach a parking lot."),
        (194, "Take a left when you see a traffic cone. Go straight passing by a semi-track and take left after you see a big log. Drive to a blue box and continue straight until you find a cardboard box next to a parking lot."),
        (133, "Take right at a traffic cone, and go straight until you reach a square with a big tree. Take right next and go straight until you find a baby stroller. Take left and right and look for an intersection."),
    ]
    all_routes_gt = [
        [180,188, 224,220, 216],
        [215, 220, 226, 194, 134, 131],
        [63,75,78,149,157,165],
        [160,157, 149,202,38,45,50],
        [61, 78, 121],
        [219, 131, 182],
        [186, 15, 205, 44],
        [75, 52, 62, 69, 216, 240],
        [194, 134, 131, 220, 240],
        [133,138,230,216,63],
    ]
    landmarks_cache = eval("[['a stop sign', 'a traffic cone', 'a blue box', 'a baby stroller'], ['a blue box', 'a traffic cone', 'a semi-truck', 'a big log'], ['a traffic cone', 'a cardboard box', 'a parking lot', 'a picnic table', 'a stop sign'], ['a picnic table', 'a square with a large tree', 'another picnic table', 'a parking lot'], ['a traffic cone', 'a parking lot', 'a lawn', 'a blue box'], ['a blue box', 'a big log', 'a traffic cone', 'a parking lot'], ['a traffic cone', 'a square with a tree', 'a cardboard box', 'a picnic table'], ['a picnic table', 'a street', 'an orange trailer', 'an intersection', 'a traffic cone', 'a baby stroller', 'a parking lot'], ['a traffic cone', 'a semi-track', 'a big log', 'a blue box', 'a cardboard box', 'a parking lot'], ['a traffic cone', 'a square with a big tree', 'a baby stroller', 'an intersection']]")

In [ ]:
url = f'https://drive.google.com/uc?id={graph_file_gdrive_id}'
gdown.download(url, "graph.pkl")
graph = NavigationGraph("graph.pkl")

In [ ]:
graph.vert_count

In [ ]:
# in order not to query OpenAI API, which incurse cost, we have cached the output of landmark query.
# If you want to rerun it, please setup your API KEY in env variable:
# e.g.: OPENAI_API_KEY=sk-[real api key here] jupyter notebook
# and uncomment the line below:
# landmarks_cache = [None] * len(all_routes)

## Visualize

In [17]:
display(Javascript("require.config({paths: {d3: 'https://d3js.org/d3.v5.min'}});"))
display(Javascript(filename=lm_nav.__path__[0]+"/bgraph.js"))
display(HTML(filename=lm_nav.__path__[0]+"/base.css.html"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def draw_bgraph(verticies, edges, traversal=[], supplemenatry_data={}, width=600, height=400):
    display(Javascript("""
        (function(element){
            require(['bgraph'], function(bgraph) {
                bgraph(element.get(0), %s, %s, %s, %s, %d, %d);
            });
        })(element);
    """ % (json.dumps(verticies), json.dumps(edges), json.dumps(traversal), json.dumps(supplemenatry_data), width, height)))

In [ ]:
vis_ver, vis_edge = graph.json_repr_for_visualization(1000)

In [ ]:
# This should open an interactive visualization in Javascript, it can take about 20 seconds to load.
draw_bgraph(vis_ver,vis_edge, width=1300, height=1300)

## Compute routes

In [ ]:
alpha = 0.0002

In [ ]:
# This step can take a few minutes.
all_results = [pipeline.full_pipeline(graph, start_node=start, instructions=description, alpha=alpha) if cached_landmarks is None else pipeline.full_pipeline(graph, start_node=start, landmarks=cached_landmarks, alpha=alpha) for ((start, description), cached_landmarks) in zip(all_routes, landmarks_cache)]

In [ ]:
# Set inx = 0 .. 9 to inpect result on a particular example.
inx = 0
print(all_routes[inx])
draw_bgraph(vis_ver,vis_edge, all_results[inx]["walk"], all_results[inx]["supplementary_data"], width=1300, height=1300)

In [ ]:
descriptions_with_walks = [(route_input[1], [a[0] for a in route_output["walk"]])for route_input, route_output in zip(all_routes, all_results)]

## Measure route distances and efficiency

### Floyd-Warschal algorithm

In [ ]:
distance = np.zeros((graph.vert_count,graph.vert_count))
distance.fill(1e9)

for i in range(graph.vert_count):
    distance[i,i] = 0
    
for u,v in graph._graph.edges():
    d = np.linalg.norm(graph._pos[u] - graph._pos[v])
    distance[u, v] = d
    distance[v, u] = d
    
for k in range(graph.vert_count):
    for i in range(graph.vert_count):
        for j in range(graph.vert_count):
            if distance[i,j] > distance[i,k] + distance[k,j]:
                distance[i,j] = distance[i,k] + distance[k,j]

In [ ]:
def path_length(path, distance):
    prev = None
    res = 0.
    for i in path:
        if prev is not None and i != prev:
            res += distance[prev,i]
        prev = i
    return res

In [ ]:
path_length(descriptions_with_walks[3][1], distance)

In [ ]:
def image_to_html(img):
    b64 = str(base64.b64encode(img))[2:-1]
    return f'<img src="data:image/png;base64,{b64}" />'

def display_route_landmarks(result):
    landmarks = result["landmarks"]
    path = result["walk"]
    landmarks_on_path = [f for f,s in path if s == -1]
    assert(len(landmarks) == len(landmarks_on_path))
    html_code = ""
    for landmark, node in zip(landmarks, landmarks_on_path):
        html_code += f"<h2>{landmark}</h2>"
        html_code += image_to_html(graph._images[node][0])
        html_code += image_to_html(graph._images[node][1])
    return HTML(html_code)

In [ ]:
# This method displays landmarks assigned on a route.
display_route_landmarks(all_results[4])

## Measure planning efficiency

In [ ]:
walk_with_data = []
for i, r in enumerate(all_results):
    walk = [a[0] for a in r["walk"]]
    walk_with_data.append({"walk": walk, "d_planning": path_length(walk, distance), "dh": path_length(all_routes_gt[i], distance), "success": True, "description": all_routes[i][1]})

if use_large_graph:
    walk_with_data[2]["success"] = False
    walk_with_data[5]["success"] = False
else:
    walk_with_data[7]["success"] = False

In [ ]:
# Planning Efficiency
l2 = [min(1,r["dh"]/r["d_planning"]) for r in walk_with_data]
np.average([l2[i] for i in range(len(l2)) if walk_with_data[i]["success"]])

# Max likelihood experiments

In [ ]:
def max_likelihood_selection(start, result, distance):
    dist = 0.
    prev = start
    landmarks = []
    for i in range(len(result["landmarks"])):
        current = np.argmax(result["similarity_matrix"][:,i])
        dist += distance[prev, current]
        prev = current
        landmarks.append(current)
    return dist, landmarks

In [ ]:
max_likelihood_paths = [max_likelihood_selection(all_routes[i][0], all_results[i], distance) for i in range(10)]

In [ ]:
inx = 1
print(all_routes[inx])
draw_bgraph(vis_ver,vis_edge, [(all_routes[inx][0], -1)]+[(int(v),-1) for v in max_likelihood_paths[inx][1]], width=1300, height=1300)

In [ ]:
l = [dh/mlp[0] for dh, mlp in zip([a["dh"] for a in walk_with_data], max_likelihood_paths)]
l

In [ ]:
np.average([l[3]]+[l[9]])

In [ ]:
inx = 8
[(v,-1) for v in max_likelihood_paths[inx][1]]
display_route_landmarks({"landmarks": all_results[inx]["landmarks"], "walk": [(v,-1) for v in max_likelihood_paths[inx][1]]})